In [6]:

#main imports 
import os
import pandas as pd
import numpy as np

# plotting imports 
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

# stats imports 
import scipy.stats as stats

#the following lines prevents pandas from giving unecessary errors and increases the number of rows you can view 
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows',100)

os.chdir('C:\\Users\\brett\\sparklyRGT\\sparklyRGT_tutorial')
import sparklyRGT as rgt

I am being executed!


C:\Users\brett\anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.12, the latest is 0.5.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [7]:
os.chdir('..\\data')

fnames = ['BH09_raw-free_S1-5_corrected.xlsx','CH02_corrected.xlsx',
          'NA01_raw_free-choice_S8-18.xlsx',"CH01_corrected.xlsx"]


df = rgt.load_multiple_data(fnames, reset_sessions = True)

#for plotting: 
title = 'Cue manipulation' 

In [8]:
df = df.replace(to_replace = {'MSN':
                         {'LossrGT_A-losscue_v1': 'outcomeRGT_A',
                        'LossrGT_B-losscue_v1': 'outcomeRGT_B',
                         'AnarchyrGT_A-losscue_v6':'RandomRGT_A',
                        'AnarchyrGT_B-losscue_v6': 'RandomRGT_B',
                         'MisrGT_A-cue':'RevRGT_A',
                         'MisrGT_B-cue':'RevRGT_B',
                         'RevRGT_A-cue':'RevRGT_A',
                         'RevRGT_B-cue':'RevRGT_B',
                         'LossRGT_A':'lossRGT_A'
                        }})

In [61]:
def get_win_stay(df, subs, sess):
    df2 = pd.DataFrame(columns = ['Subject','Session','Win-stay_risk','Win-stay_optimal'] )
    for sub in subs:
        for num in sess:
            df1 = df.loc[(df.Subject == sub) & (df.Session == num)]
            if df1.empty:
                df_temp = pd.DataFrame(data = {'Subject': sub, 'Session': num,
                                               'Win-stay_risk': np.nan,
                                           'Win-stay_optimal': np.nan}, index = [0])
                df2 = df2.append(df_temp, ignore_index = True)
            else:
                trials = int(df1.Trial.max())
                risky_win_stay = 0
                optimal_win_stay = 0
                for i in range(1,len(df1)):
                    prev_trial = df1.iloc[i-1, :]
                    curr_trial = df1.iloc[i, :]
                    if curr_trial.risky_choice == 1:
                        if prev_trial.risky_choice == 1 and prev_trial.Rewarded == 1:
                            risky_win_stay += 1 
                    elif curr_trial.risky_choice == 0:
                        if prev_trial.risky_choice == 0 and prev_trial.Rewarded == 1:
                            optimal_win_stay += 1
                df_temp = pd.DataFrame(data = {'Subject': sub, 'Session': num, 
                                               'Win-stay_risk': risky_win_stay/trials*100,
                                               'Win-stay_optimal': optimal_win_stay/trials*100}, index = [0])
                df2 = df2.append(df_temp, ignore_index = True)
    return df2

def get_lose_shift(df, subs, sess):
    df2 = pd.DataFrame(columns = ['Lose-shift_risk','Lose-shift_optimal'] )
    for sub in subs:
        for num in sess:
            df1 = df.loc[(df.Subject == sub) & (df.Session == num)]
            if df1.empty:
                df_temp = pd.DataFrame(data = {'Lose-shift_risk': np.nan,
                                           'Lose-shift_optimal': np.nan}, index = [0])
                df2 = df2.append(df_temp, ignore_index = True)
            else:
                trials = int(df1.Trial.max())
                risky_lose_shift = 0
                optimal_lose_shift = 0
                for i in range(1,len(df1)):
                    prev_trial = df1.iloc[i-1, :]
                    curr_trial = df1.iloc[i, :]
                    if curr_trial.risky_choice == 0:
                        if prev_trial.risky_choice == 1 and prev_trial.Rewarded == 0:
                            risky_lose_shift += 1 
                    elif curr_trial.risky_choice == 1:
                        if prev_trial.risky_choice == 0 and prev_trial.Rewarded == 0:
                            optimal_lose_shift += 1
                df_temp = pd.DataFrame(data = {'Lose-shift_risk': risky_lose_shift/trials*100,
                                               'Lose-shift_optimal': optimal_lose_shift/trials*100}, index = [0])
                df2 = df2.append(df_temp, ignore_index = True)
    return df2

def get_lose_stay(df, subs, sess):
    df2 = pd.DataFrame(columns = ['Lose-stay_risk','Lose-stay_optimal'] )
    for sub in subs:
        for num in sess:
            df1 = df.loc[(df.Subject == sub) & (df.Session == num)]
            if df1.empty:
                df_temp = pd.DataFrame(data = {'Lose-stay_risk': np.nan,
                                           'Lose-stay_optimal': np.nan}, index = [0])
                df2 = df2.append(df_temp, ignore_index = True)
            else:
                trials = int(df1.Trial.max())
                risky_lose_stay = 0
                optimal_lose_stay = 0
                for i in range(1,len(df1)):
                    prev_trial = df1.iloc[i-1, :]
                    curr_trial = df1.iloc[i, :]
                    if curr_trial.risky_choice == 1:
                        if prev_trial.risky_choice == 1 and prev_trial.Rewarded == 0:
                            risky_lose_stay += 1 
                    elif curr_trial.risky_choice == 0:
                        if prev_trial.risky_choice == 0 and prev_trial.Rewarded == 0:
                            optimal_lose_stay += 1
                df_temp = pd.DataFrame(data = {'Lose-stay_risk': risky_lose_stay/trials*100,
                                               'Lose-stay_optimal': optimal_lose_stay/trials*100}, index = [0])
                df2 = df2.append(df_temp, ignore_index = True)
    return df2

def average_sessions(df):
    df = df.drop("Session", 1)
    averages = df.groupby(['Subject']).mean()
    return averages
                 


In [63]:
def get_stay_shift(df, sub, sess):
    df2 = rgt.get_choices(df)

    #this is to simplify P1-P4 to designate each trial as "risky choice" (== 1 in column) or "optimal choice"
    df2 = df2.loc[df2.option != 0]
    df2['risky_choice'] = np.where(df2['option']>2,1,0)
    df2.reset_index(drop = True, inplace = True)

    df3 = get_win_stay(df2, sub, sess)
    df4 = get_lose_shift(df2, sub, sess)
    df5 = get_lose_stay(df2, sub, sess)
    df6 = pd.concat([df3,df4,df5], axis = 1)
    df6.sort_values(by = ['Subject','Session'], inplace = True, ignore_index = True)
    
    df_averages = average_sessions(df6)
    return df_averages


In [19]:
data = [[1,1, 10, 12,13,14], [1,2, 10, 12,13,14], [1,3, 10, 12,13,14], [2,1, 1, 2,3,4],[2,2, 1, 2,3,4],[2,3, 1, 2,3,4]]
df_test = pd.DataFrame(data, columns=['Subject', 'Session', 'Win-stay_risk', 'Win-stay_optimal', 'Lose-shift_risk', 'Lose-shift_optimal'])
df_test

,Subject,Session,Win-stay_risk,Win-stay_optimal,Lose-shift_risk,Lose-shift_optimal
0,1,1,10,12,13,14
1,1,2,10,12,13,14
2,1,3,10,12,13,14
3,2,1,1,2,3,4
4,2,2,1,2,3,4
5,2,3,1,2,3,4


In [64]:
df_averages_sess1 = get_stay_shift(df, df.Subject.unique(), [1])

In [65]:
df_averages_sess1

,Win-stay_risk,Win-stay_optimal,Lose-shift_risk,Lose-shift_optimal,Lose-stay_risk,Lose-stay_optimal
Subject,,,,,,
101,0.000000,71.153846,4.807692,0.961538,0.000000,12.500000
102,0.980392,61.764706,6.862745,1.960784,0.980392,9.803922
103,43.859649,0.000000,0.000000,0.000000,47.368421,0.000000
104,4.000000,45.333333,8.000000,1.333333,5.333333,8.000000
105,5.263158,31.578947,14.035088,1.754386,5.263158,8.771930
...,...,...,...,...,...,...
444,13.043478,19.565217,8.695652,2.173913,19.565217,8.695652
445,0.000000,75.247525,0.990099,0.000000,0.000000,10.891089
446,17.777778,13.333333,22.222222,6.666667,8.888889,0.000000


In [67]:
df_averages_sess1['task'] = 0
subs = df_averages.index

for sub in subs:
    if 'cue' in list(df.loc[df['Subject']==sub]['MSN'])[0]:
        df_averages_sess1.at[sub,'task'] = 1
    if 'Rev' in list(df.loc[df['Subject']==sub]['MSN'])[0]:
        df_averages_sess1.at[sub,'task'] = 2
    if 'outcome' in list(df.loc[df['Subject']==sub]['MSN'])[0]:
        df_averages_sess1.at[sub,'task'] = 3
    if 'Random' in list(df.loc[df['Subject']==sub]['MSN'])[0]:
        df_averages_sess1.at[sub,'task'] = 4
    if 'loss' in list(df.loc[df['Subject']==sub]['MSN'])[0]:
        df_averages_sess1.at[sub,'task'] = 5
    
        
df_averages_sess1

,Win-stay_risk,Win-stay_optimal,Lose-shift_risk,Lose-shift_optimal,Lose-stay_risk,Lose-stay_optimal,task
Subject,,,,,,,
101,0.000000,71.153846,4.807692,0.961538,0.000000,12.500000,5
102,0.980392,61.764706,6.862745,1.960784,0.980392,9.803922,5
103,43.859649,0.000000,0.000000,0.000000,47.368421,0.000000,2
104,4.000000,45.333333,8.000000,1.333333,5.333333,8.000000,3
105,5.263158,31.578947,14.035088,1.754386,5.263158,8.771930,5
...,...,...,...,...,...,...,...
444,13.043478,19.565217,8.695652,2.173913,19.565217,8.695652,2
445,0.000000,75.247525,0.990099,0.000000,0.000000,10.891089,2
446,17.777778,13.333333,22.222222,6.666667,8.888889,0.000000,2


In [68]:
df_averages_sess1.to_excel('rgt-variants_sess1_winstay-loseshift.xlsx')